**New Approach using the Sentiment Similarity BERT**
<br><br>

using an older implementation that is dependent on keras 3.

In [1]:
# change this to pip install transformers version 2.11.0
!pip install -q transformers==2.11.0
!pip list | grep accelerate
!pip list | grep datasets

!pip list | grep huggingface_hub
!pip list | grep keras

!pip install -q -U langchain-nomic langchain_community unstructured tiktoken langchainhub chromadb langchain langgraph gpt4all

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 674.8/674.8 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 20.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)
accelerate                         1.5.2
tensorflow-datasets                4.9.8
vega-datasets                      0.9.0
keras               

In [2]:
# code cell one
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import tensorflow.keras as tf
import tensorflow as tf
from keras.layers import Lambda
import transformers

from huggingface_hub import from_pretrained_keras
# import bitsandbytes as bnb
import torch
import random

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain.docstore.document import Document
from langchain_community.document_loaders import UnstructuredMarkdownLoader



# from matplotlib.ticker import MultipleLocator
# from sklearn.pipeline import Pipeline
# from sklearn.datasets import fetch_openml
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import classification_report
# from sklearn.preprocessing import MinMaxScaler

# from sklearn.decomposition import LatentDirichletAllocation
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer

# import sklearn
# sklearn.__version__



In [ ]:
# ! pip install teapotai
# !pip install ollama
! ollama pull

/bin/bash: line 1: ollama: command not found


**Semantic Similarity with BERT**
<br><br>
bring in functions from the main repo to use the model, and check out the SNLI dataset. [source](https://keras.io/examples/nlp/semantic_similarity_with_bert/)
<br><br>
we will be using the keras.io implementation which trains off of the SNLI (stanford natural language interface) corpus and employes transformers to assess/predict sentence semantic similarity. <br>
we can bring in the SNLI dataset like so: <br>


In [ ]:
!curl -LO https://raw.githubusercontent.com/MohamadMerchant/SNLI/master/data.tar.gz
!tar -xvzf data.tar.gz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.1M  100 11.1M    0     0  6694k      0  0:00:01  0:00:01 --:--:-- 6692k
SNLI_Corpus/
SNLI_Corpus/snli_1.0_dev.csv
SNLI_Corpus/snli_1.0_train.csv
SNLI_Corpus/snli_1.0_test.csv


In [ ]:
train_df_SNLI = pd.read_csv("SNLI_Corpus/snli_1.0_train.csv", nrows=100000)
valid_df_SNLI = pd.read_csv("SNLI_Corpus/snli_1.0_dev.csv")
test_df_SNLI = pd.read_csv("SNLI_Corpus/snli_1.0_test.csv")

print(f"shape of training: {train_df_SNLI.shape} \nshape of validation: {valid_df_SNLI.shape} \nshape of testing: {test_df_SNLI.shape}")
print(f"data types: {train_df_SNLI.dtypes} \n\nexample: \n{train_df_SNLI.iloc[0]}")

shape of training: (100000, 3) 
shape of validation: (10000, 3) 
shape of testing: (10000, 3)
data types: similarity    object
sentence1     object
sentence2     object
dtype: object 

example: 
similarity                                              neutral
sentence1     A person on a horse jumps over a broken down a...
sentence2     A person is training his horse for a competition.
Name: 0, dtype: object


for each record, we have a similarity label:<br>
similarity: neutral <br><br>

and 2 sentences that demonstrate the relationship:<br>
sentence 1: ... <br>
sentence 2: ...
<br><br>

now we will prep the data to train our BERT.

In [6]:
# convert to 1 hot
# 0 - contradiction, 1 - entailment, 2 - neutral

def make_1hot(df, colname):
  # adapt to take in our custom data
  df["label"] = df[colname].apply(
      lambda x: 0 if x == "contradiction" else 1 if x == "entailment" else 2
  )
  y_df = tf.keras.utils.to_categorical(df.label, num_classes=3)
  return df, y_df



In [12]:
# now we bring in our custom data

agora_df = pd.read_csv("agora_sentences_labeled.csv")
agora = agora_df[["sentence1", "sentence2", "pred"]]
#agora.head()

agora_train = agora.sample(frac=0.8, random_state=25).reset_index(drop=True)
agora_test = agora.drop(agora_train.index).reset_index(drop=True)

agora_train, y_agora_train = make_1hot(agora_train, "pred")
agora_test, y_agora_test = make_1hot(agora_test, "pred")

y_agora_train


array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

**process and batch data**
<br><br>
now, we are going to use classes to manage the data and generate our batches, since we have the 2 sentances per record and label, we have to employ a special lil method for this. <br>

we will need our returned generated batch data to include the input_ids, attention_masks, token_type_ids, labels, and lastly, boolean to indicate if we are using the dataGenerator for testing/validation.

In [3]:
### need this func to pass data to do predictions ###

# initialize global vars
max_length = 128
batch_size = 32
epochs = 2

labels = ["contradiction", "entailment", "neutral"]

# BERT generator for batching and processesing data.

class DataGenerator(tf.keras.utils.Sequence):
  def __init__(self, sentence_pairs, labels, batch_size=batch_size, shuffle=True, include_targets=True):
    self.sentence_pairs = sentence_pairs
    self.labels = labels
    self.shuffle = shuffle
    self.batch_size = batch_size
    self.include_targets = include_targets
    # load in bert-base-uncased pretrained BERT tokenizer model.
    # transformers.BertTokenizer.
    self.tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
    self.indexes = np.arange(len(self.sentence_pairs))
    self.on_epoch_end()

  def __len__(self):
    # returns int number of batches per epoch.
    return len(self.sentence_pairs) // self.batch_size

  def __getitem__(self, idx):
    # gets batch of indexes of current data
    indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
    sentence_pairs = self.sentence_pairs[indexes]

    # batch_encode_plus will batch the two sentences together using the
    # [SEP] token to seperate them.
    encoded = self.tokenizer.batch_encode_plus(
        sentence_pairs.tolist(),
        add_special_tokens=True,
        max_length=max_length,
        return_attention_mask=True,
        return_token_type_ids=True,
        pad_to_max_length=True,
        return_tensors="tf",
      )
    # store input_ids, att_masks, and token_types
    input_ids = np.array(encoded["input_ids"], dtype="int32")
    att_masks = np.array(encoded["attention_mask"], dtype="int32")
    token_types = np.array(encoded["token_type_ids"], dtype="int32")

    # TRUE if used for training/validation
    if self.include_targets:
      labels = np.array(self.labels[indexes], dtype="int32")
      return [input_ids, att_masks, token_types], labels
    else:
      return [input_ids, att_masks, token_types]

  def on_epoch_end(self):
    # if shuffle is true, then will shuffle indx after each epoch.
    if self.shuffle:
      np.random.RandomState(42).shuffle(self.indexes)


[this might help with updating this call below](https://huggingface.co/docs/transformers/en/training)

**TF Strategy to build model** <br><br>

now that we have our method for handling input data (DataGenerator), we can build the BERT model we'll be passing it to. <br>

for more on [tensor flow distribution strategy](https://www.tensorflow.org/api_docs/python/tf/distribute/MirroredStrategy) <br><br><br>

steps!<br>
run all of the cells for the data.<br>
then run the cells for the model, this one above will fail, but its like needed for some reason.<br>
then process the data via the DataGenerator() call. <br>
instantiate the model from the pretrained_keras repo. <br>
COMPILE IT AND PRINT SUMMARY! <br>
now you are good to use the model for comparisons. <br><br>

for more on potentially updating this model for fine-tuning look at this [tensor flow documentation](https://www.tensorflow.org/tfmodels/nlp/fine_tune_bert) <br>

In [15]:
# action time!!!


agora_train_data = DataGenerator(
    agora_test[["sentence1", "sentence2"]].values.astype("str"),
    y_agora_train,
    batch_size=batch_size,
    shuffle=True,
)

agora_test_data = DataGenerator(
    agora_test[["sentence1", "sentence2"]].values.astype("str"),
    y_agora_test,
    batch_size=batch_size,
    shuffle=False,
)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [4]:
model = from_pretrained_keras("keras-io/bert-semantic-similarity")

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

(…)events.1657167291.672ead9668f3.3159.1.v2:   0%|          | 0.00/6.47k [00:00<?, ?B/s]

keras_metadata.pb:   0%|          | 0.00/169k [00:00<?, ?B/s]

(…)events.1657166913.672ead9668f3.3159.0.v2:   0%|          | 0.00/2.97M [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

saved_model.pb:   0%|          | 0.00/7.88M [00:00<?, ?B/s]

model.png:   0%|          | 0.00/30.0k [00:00<?, ?B/s]

variables.data-00000-of-00001:   0%|          | 0.00/440M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

variables.index:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

In [5]:

model.compile()
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 attention_masks (InputLaye  [(None, 128)]                0         []                            
 r)                                                                                               
                                                                                                  
 token_type_ids (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                                                              

In [ ]:
# from tensorflow.keras.optimizers import Adam
# we can try to compile with:
# model.compile(optimizer=Adam(3e-5))
# then, try to fit the model to the custom dataset, if we have time to come up with one.
# after model.fit(dataset), we cna try predicting stuff.


# agora_model = model.fit(
#     agora_train_data,
#     steps_per_epoch=train_steps_per_epoch,
#     validation_data=agora_test_data,
#     validation_steps=validation_steps_per_epoch,
#     epochs=epochs,
#     use_multiprocessing=True,
#     workers=1,
# )

In [6]:
# call check similarity on small chunks like sentence length (128 max)
# will give score.
# need to use this func to

def check_similarity(sentence1, sentence2):
  s_pairs = np.array([[str(sentence1), str(sentence2)]])
  gen_data = DataGenerator(
      s_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
  )

  proba = model.predict(gen_data[0])[0]
  ind = np.argmax(proba)
  proba = f"{proba[ind]: .3f}%"
  pred = labels[ind]
  return pred, proba


In [7]:
# test it out, works pretty good!
s1 = "The cat sits on the mat"
s2 = "The cat squats on the mat"
check_similarity(s1, s2)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2700: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


1/1 [==============================] - 3s 3s/step


('entailment', ' 0.959%')

**Lets bring in the data** <br><br>

going to use the agora data.

In [8]:
testdf = pd.read_csv("perplexity_content.csv")
testdf.head()



,Unnamed: 0,user,golden
0,gov0,We will conduct regular assessments to evaluat...,and improve performance can also increase leve...
1,gov1,Clear responsibilities will be designated with...,Issues that merit further consideration and re...
2,map0,Maintain a public AI inventory cataloging acti...,"such as predictions, recommenda- tions, or dec..."
3,map1,This policy is designed to be a living documen...,"(e.g., UX/UI design), governance experts, data..."
4,mea0,We will regularly audit AI systems for biases ...,the AI lifecycle can en- hance opportunities f...


**comparing sentences** <br><br>

since we have a limited bit of room to work with, we have to split up the text docs into more manegable peices.<br>

here, we chunck the text docs from agora and the ai policies. then we compare them, getting a few score to which we will combine together to calculate the over all score using lawrence's method.<br><br>

1. govern: 25% <br>
2. map: 20% <br>
3. Measure: 30% <br>
4. Manage: 25% <br> <br>

given the 3 labels available: <br>
1. Contradiction <br>
2. Entailment <br>
3. Neutral <br><br>

we can use to calculate sub scores as such: <br>
1. Contradiction: will be negitive. <br>
2. Entailment: will be positive. <br>
3. Neutral: will be the varience +/-. <br><br>

so for example, after we normilaize the counts: <br>c = -0.25, e = 0.625, n = 0.125 <br>
score = 35.5% +/- 12.5%

In [9]:
# test to see if things are working
para1 = testdf.at[0, 'user']
para2 = testdf.at[0, 'golden']

def chunk_it(text, chunk_size=148):
  return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

p1_chunks = chunk_it(para1)
p2_chunks = chunk_it(para2)

print(f"p1 len: {len(p1_chunks)} \np2 len: {len(p2_chunks)}")

p1 len: 3 
p2 len: 13


In [10]:
num_pairs = min(len(p1_chunks), len(p2_chunks))
results = []

for i in range(num_pairs):
  p1 = p1_chunks[i]
  p2 = p2_chunks[i]

  pred, proba = check_similarity(p1, p2)
  results.append([p1, p2, pred, proba])
  print(f"{i}: \npred: {pred} \nproba: {proba}\n")



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 58ms/step


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2700: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0: 
pred: neutral 
proba:  0.938%

1/1 [==============================] - 0s 59ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1: 
pred: neutral 
proba:  0.953%

1/1 [==============================] - 0s 59ms/step
2: 
pred: neutral 
proba:  0.947%



In [ ]:
n = 1

print(f"p1: \n{p1_chunks[n]} \np2: \n{p2_chunks[n]}")


p1: 
ision-making 
p2: 
ibility. • Higher degree of difficulty in predicting failure modes for emergent properties of large-scale pre-trained models. • Privacy risk due to 


In [11]:

# going to make the boiler plate code for the comparision function.
# get the text items, prob going to be 1-3 items for both the
# proposed doc and the golden standard docs.

# note, will only do one topic at a time so, only govern or only map.

cats = ["governance", "map", "measure", "manage"]

ugov = [testdf.at[0, 'user'], testdf.at[1, 'user']]
ggov = [testdf.at[0, 'golden'], testdf.at[1, 'golden']]

umap = [testdf.at[2, 'user'], testdf.at[3, 'user']]
gmap = [testdf.at[2, 'golden'], testdf.at[3, 'golden']]

umea = [testdf.at[4, 'user'], testdf.at[5, 'user']]
gmea = [testdf.at[4, 'golden'], testdf.at[5, 'golden']]

uman = [testdf.at[6, 'user'], testdf.at[7, 'user']]
gman = [testdf.at[6, 'golden'], testdf.at[7, 'golden']]


def score_per_cat(proposed, golden, cat):
  raw_scores = []

  for i in range(len(proposed)):
    proposed_chunks = chunk_it(proposed[i])
    golden_chunks = chunk_it(golden[i])

    num_pairs = min(len(proposed_chunks), len(golden_chunks))
    results = []

    for j in range(num_pairs):
      p1 = proposed_chunks[j]
      p2 = golden_chunks[j]

      pred, proba = check_similarity(p1, p2)
      raw_scores.append(pred)
      #print(f"{i}.{j}: \npred: {pred} \nproba: {proba}\n")

  counts = pd.Series(raw_scores).value_counts(normalize=True)
  print(counts)
  my_score = ""
  try:
    if 'entailment' in counts.index and 'contradiction' in counts.index and 'neutral' in counts.index:
      score = (counts.loc['entailment'] - counts.loc['contradiction'])*100
      my_score = f"score: {score} +/- {counts.loc['neutral']*100}"
    elif 'entailment' in counts.index and 'contradiction' in counts.index:
      score = (counts.loc['entailment'] - counts.loc['contradiction'])*100
      my_score = f"score: {score}"
    elif 'entailment' in counts.index and 'neutral' in counts.index:
      score = (counts.loc['entailment'])*100
      my_score = f"score: {score} +/- {counts.loc['neutral']*100}"
    elif 'contradiction' in counts.index and 'neutral' in counts.index:
      score = (counts.loc['contradiction'])*100
      my_score = f"score: {score} +/- {counts.loc['neutral']*100}"
    elif 'neutral' in counts.index:
      my_score = f"score: +/-{counts.loc['neutral']*100}"
  except:
    print("missing value")
    my_score = counts
  print(my_score)
  return (cat, my_score)

# gov_score = score_per_cat(ugov, ggov, cats[0])
# map_score = score_per_cat(umap, gmap, cats[1])
# mea_score = score_per_cat(umea, gmea, cats[2])
# man_score = score_per_cat(uman, gman, cats[3])


In [12]:
# scores
gov_score = score_per_cat(ugov, ggov, cats[0])
map_score = score_per_cat(umap, gmap, cats[1])
mea_score = score_per_cat(umea, gmea, cats[2])
man_score = score_per_cat(uman, gman, cats[3])

final_score = pd.DataFrame([gov_score, map_score, mea_score, man_score], columns=['category', 'score'])
final_score.head()


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 58ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 58ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


...

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 75ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 0s 86ms/step
entailment    0.857143
neutral       0.142857
Name: proportion, dtype: float64
score: 85.71428571428571 +/- 14.285714285714285


,category,score
0,governance,score: 33.33333333333333 +/- 66.66666666666666
1,map,score: 33.333333333333336 +/- 33.33333333333333
2,measure,score: 80.0 +/- 20.0
3,manage,score: 85.71428571428571 +/- 14.285714285714285


In [13]:

final_score.to_csv('perplexity_scores.csv')



In [ ]:
# counts = pd.Series(raw_scores).value_counts(normalize=True)
# print(counts)
# score = (counts.loc['entailment'] - counts.loc['contradiction'])*100
# print(f"score: {score} +/- {counts.loc['neutral']*100}")

entailment       0.592593
neutral          0.333333
contradiction    0.074074
Name: proportion, dtype: float64
score: 51.85185185185185 +/- 33.33333333333333


In [ ]:

# agora_documents_main = pd.read_csv('AGORA_data_cleaned.csv')
# # agora_documents_main.head()
# agora_corpus = pd.DataFrame()
# agora_corpus['text'] = agora_documents_main['text']

# ai_policies_main = pd.read_csv('clean_final_df.csv')
# # ai_policies_main.head()

# ai_policies_corpus = pd.DataFrame()
# ai_policies_corpus['text'] = ai_policies_main['text']
# # ai_policies_corpus.tail()


# corpus = pd.DataFrame()
# corpus = pd.concat([agora_corpus, ai_policies_corpus], ignore_index=True)

# corpus.head()


,text
0,H. R. 4611\n\n\nTo amend the Federal Election...
1,Section 1. Chapter 5 of Title 6 of the Rules o...
2,Document Type: Regulation\nNumber: 4.30q\nEffe...
3,Overview: Data privacy and security are core v...
4,PROPOSAL FOR A COUNCIL RESOLUTION establishing...


**getting some labels** <br><br>
bringing some sentences.

In [10]:

agora_sentences = pd.read_csv("agora_sentences.csv")

for i in range(agora_sentences.shape[0]):
  pred, proba = check_similarity(agora_sentences.at[i, 'sentence1'], agora_sentences.at[i, 'sentence2'])
  agora_sentences.at[i, 'pred'] = pred
  agora_sentences.at[i, 'proba'] = proba


agora_sentences.head()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2700: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs 

1/1 [==============================] - 0s 87ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2700: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs 

1/1 [==============================] - 0s 81ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


...

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


1/1 [==============================] - 0s 64ms/step


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


1/1 [==============================] - 0s 57ms/step


,Unnamed: 0,sentence1,sentence2,pred,proba
0,0,h r to amend the federal election campaign act...,amends the federal election campaign act to pr...,neutral,0.75%
1,1,section chapter of title of the rules of th...,requires new york city employers to conduct an...,neutral,1.00%
2,2,document type regulationnumber qeffective revi...,requires boises city departments and contracto...,neutral,0.84%
3,3,overview data privacy and security are core va...,requires long beach to ensure transparency acc...,entailment,0.82%
4,4,proposal for a council resolution establishing...,establishes a study commission to review and r...,neutral,0.94%


In [11]:

agora_sentences.to_csv('agora_sentences_labeled.csv')

